In [1]:
import pandas as pd
df = pd.read_csv("./temp_bq_data/hackathon_data.csv")
df1 = df.iloc[:len(df)//2]
df2 = df.iloc[len(df)//2:]
len(df1)+len(df2)
len(df)

9678979

9678979

In [2]:
basket = (df\
          .groupby(['customer_id', 'product_id'])['product_quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('customer_id'))

In [3]:
basket.head()

product_id,1120112,1120224,1120336,1120448,1120560,1120672,1120784,1120896,1121008,1121120,...,4193392,4193952,4194064,4194288,4195184,4195744,4201232,4201792,4205488,4214000
customer_id,,,,,,,,,,,,,,,,,,,,,
1121904,0.0,17.0,52.0,0.0,0.0,1.0,0.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1122016,108.0,262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1122128,0.0,0.0,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1122352,0.0,98.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1123248,0.0,519.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
#basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [5]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
#frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets = apriori(basket_sets, min_support=0.3, use_colnames=True, low_memory=True)

In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1120112),(1120224),0.506733,0.636933,0.383233,0.756282,1.187380,0.060478,1.489700
1,(1120224),(1120112),0.636933,0.506733,0.383233,0.601685,1.187380,0.060478,1.238384
2,(1120112),(1681904),0.506733,0.666133,0.338067,0.667149,1.001525,0.000515,1.003052
3,(1681904),(1120112),0.666133,0.506733,0.338067,0.507506,1.001525,0.000515,1.001569
4,(1120112),(1709008),0.506733,0.736433,0.409200,0.807525,1.096536,0.036025,1.369358


In [7]:
rules['antecedents1'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents1'] = rules['consequents'].apply(lambda x: list(x)[0])

In [8]:
import json
with open('top_3.json', 'r') as file:
    data = json.load(file)

In [9]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents1,consequents1
0,(1120112),(1120224),0.506733,0.636933,0.383233,0.756282,1.187380,0.060478,1.489700,1120112,1120224
1,(1120224),(1120112),0.636933,0.506733,0.383233,0.601685,1.187380,0.060478,1.238384,1120224,1120112
2,(1120112),(1681904),0.506733,0.666133,0.338067,0.667149,1.001525,0.000515,1.003052,1120112,1681904
3,(1681904),(1120112),0.666133,0.506733,0.338067,0.507506,1.001525,0.000515,1.001569,1681904,1120112
4,(1120112),(1709008),0.506733,0.736433,0.409200,0.807525,1.096536,0.036025,1.369358,1120112,1709008
5,(1709008),(1120112),0.736433,0.506733,0.409200,0.555651,1.096536,0.036025,1.110089,1709008,1120112
6,(1120224),(1120560),0.636933,0.365533,0.302267,0.474566,1.298283,0.069446,1.207509,1120224,1120560
7,(1120560),(1120224),0.365533,0.636933,0.302267,0.826920,1.298283,0.069446,2.097676,1120560,1120224
8,(1120224),(1379840),0.636933,0.525667,0.370533,0.581746,1.106682,0.035719,1.134079,1120224,1379840
9,(1379840),(1120224),0.525667,0.636933,0.370533,0.704883,1.106682,0.035719,1.230245,1379840,1120224


In [10]:
rules.to_csv("dump_rules.csv", index=False)

In [25]:
final_list1 = list()
for i in df['customer_id'].unique():
    reco = list()
    try:
        for j in data[str(i)]:
            for row in rules.itertuples():
                if int(j) == row[10]:
                    #print(row[10])
                    reco.append(row[11])
                    break
                elif int(j) == row[11]:
                    #print(row[11])
                    reco.append(row[10])
                    break
            else:
                reco.append('NA')
    except:
        reco.append('NA')
    final_list1.append(reco)

In [27]:
from copy import deepcopy
fin = deepcopy(final_list1)
fin2 = deepcopy(final_list1)

In [28]:
len(final_list1)
final_list1

30000

[['NA'],
 ['NA', 'NA', 'NA'],
 ['NA'],
 [1120112, 'NA', 'NA'],
 ['NA'],
 [1120112, 1120224],
 ['NA'],
 [1120224, 1120112],
 ['NA', 'NA'],
 ['NA'],
 ['NA'],
 ['NA'],
 [1120224],
 ['NA'],
 ['NA'],
 ['NA', 1120112],
 ['NA'],
 [1120112],
 [1120224, 1120112, 'NA'],
 [1120112, 1120224],
 ['NA', 1120224],
 ['NA'],
 [1120112],
 ['NA'],
 [1120224],
 ['NA', 'NA', 1379840],
 [1688064],
 [1120112],
 ['NA'],
 ['NA'],
 [1120112, 'NA', 'NA'],
 ['NA'],
 ['NA'],
 [1120224, 1120224, 'NA'],
 ['NA'],
 ['NA'],
 ['NA'],
 ['NA'],
 [1120224, 1120112, 'NA'],
 [1120112],
 ['NA'],
 ['NA'],
 [1120112],
 [1120224],
 ['NA'],
 ['NA'],
 ['NA'],
 [1120112],
 [1120112],
 [1688064],
 [1120112],
 ['NA'],
 ['NA', 'NA', 1120224],
 ['NA'],
 [1120224, 'NA'],
 ['NA'],
 ['NA'],
 [1120224],
 ['NA', 1120224, 'NA'],
 [1120112, 1120224],
 ['NA'],
 ['NA', 'NA'],
 ['NA', 'NA', 1120224],
 [1120112, 'NA'],
 [1120112, 1120224],
 ['NA'],
 ['NA'],
 [1120224, 1120112],
 ['NA'],
 ['NA'],
 ['NA'],
 ['NA'],
 ['NA', 'NA', 1120112],
 ['NA'],
 

In [29]:
cu = df['customer_id'].unique()
dic_data = dict()
for i, j in zip(cu, final_list1):
        dic_data[str(i)] = dict()
        dic_data[str(i)]["recommended_pro"] = j
dic_data

{'2698080': {'recommended_pro': ['NA']},
 '1134224': {'recommended_pro': ['NA', 'NA', 'NA']},
 '3686704': {'recommended_pro': ['NA']},
 '4045328': {'recommended_pro': [1120112, 'NA', 'NA']},
 '3568656': {'recommended_pro': ['NA']},
 '3641008': {'recommended_pro': [1120112, 1120224]},
 '4921952': {'recommended_pro': ['NA']},
 '2461312': {'recommended_pro': [1120224, 1120112]},
 '1588384': {'recommended_pro': ['NA', 'NA']},
 '3058944': {'recommended_pro': ['NA']},
 '3892112': {'recommended_pro': ['NA']},
 '3181696': {'recommended_pro': ['NA']},
 '4082176': {'recommended_pro': [1120224]},
 '2706928': {'recommended_pro': ['NA']},
 '4615296': {'recommended_pro': ['NA']},
 '1577968': {'recommended_pro': ['NA', 1120112]},
 '1504720': {'recommended_pro': ['NA']},
 '4034688': {'recommended_pro': [1120112]},
 '3580416': {'recommended_pro': [1120224, 1120112, 'NA']},
 '2371264': {'recommended_pro': [1120112, 1120224]},
 '1540448': {'recommended_pro': ['NA', 1120224]},
 '4469696': {'recommended_pr

In [31]:
import json
with open('reco_1.json', 'w') as file:
    json.dump(dic_data, file)

In [32]:
fin = final_list1
for loc, ls in enumerate(final_list1):
    final_list1[loc] = [x for x in ls if 'NA'!=x]

In [33]:
cu = df['customer_id'].unique()
dic_data2 = dict()
for i, j in zip(cu, final_list1):
        dic_data2[str(i)] = dict()
        dic_data2[str(i)]["recommended_pro"] = j
dic_data2

import json
with open('reco_2.json', 'w') as file:
    json.dump(dic_data2, file)

{'2698080': {'recommended_pro': []},
 '1134224': {'recommended_pro': []},
 '3686704': {'recommended_pro': []},
 '4045328': {'recommended_pro': [1120112]},
 '3568656': {'recommended_pro': []},
 '3641008': {'recommended_pro': [1120112, 1120224]},
 '4921952': {'recommended_pro': []},
 '2461312': {'recommended_pro': [1120224, 1120112]},
 '1588384': {'recommended_pro': []},
 '3058944': {'recommended_pro': []},
 '3892112': {'recommended_pro': []},
 '3181696': {'recommended_pro': []},
 '4082176': {'recommended_pro': [1120224]},
 '2706928': {'recommended_pro': []},
 '4615296': {'recommended_pro': []},
 '1577968': {'recommended_pro': [1120112]},
 '1504720': {'recommended_pro': []},
 '4034688': {'recommended_pro': [1120112]},
 '3580416': {'recommended_pro': [1120224, 1120112]},
 '2371264': {'recommended_pro': [1120112, 1120224]},
 '1540448': {'recommended_pro': [1120224]},
 '4469696': {'recommended_pro': []},
 '4643744': {'recommended_pro': [1120112]},
 '4275600': {'recommended_pro': []},
 '1712